In [1]:
import os

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, month, when, concat_ws
from pyspark.sql.utils import AnalysisException

spark = SparkSession.builder.master('local').getOrCreate()

In [2]:
input_files = []
for data_year in range(2015, 2025):
    file_path = f'data/{data_year}/72429793812.csv'
    if os.path.exists(file_path):
        input_files.append(file_path)
    file_path = f'data/{data_year}/99495199999.csv'
    if os.path.exists(file_path):
        input_files.append(file_path)

csv_df = spark.read.csv(input_files, sep=",", header=True)
csv_df = csv_df.withColumn('YEAR', year(col('DATE')))
csv_df = csv_df.withColumn('MONTH', month(col('DATE')))

In [3]:
csv_df.createOrReplaceTempView("WEATHER_DATA")
queried_data = spark.sql("""
    SELECT
        WEATHER_DATA.STATION,
        WEATHER_DATA.NAME AS LOCATION,
        WEATHER_DATA.YEAR,
        COUNT(WEATHER_DATA.TEMP) AS NUM_ENTRIES
    FROM WEATHER_DATA
    GROUP BY WEATHER_DATA.YEAR, WEATHER_DATA.NAME, WEATHER_DATA.STATION
    ORDER BY WEATHER_DATA.YEAR, WEATHER_DATA.STATION
""")
queried_data.show()

+-----------+--------------------+----+-----------+
|    STATION|            LOCATION|YEAR|NUM_ENTRIES|
+-----------+--------------------+----+-----------+
|72429793812|CINCINNATI MUNICI...|2015|        365|
|99495199999|SEBASTIAN INLET S...|2015|        355|
|72429793812|CINCINNATI MUNICI...|2016|        366|
|72429793812|CINCINNATI MUNICI...|2017|        365|
|99495199999|SEBASTIAN INLET S...|2017|        283|
|72429793812|CINCINNATI MUNICI...|2018|        365|
|99495199999|SEBASTIAN INLET S...|2018|        363|
|72429793812|CINCINNATI MUNICI...|2019|        365|
|99495199999|SEBASTIAN INLET S...|2019|        345|
|72429793812|CINCINNATI MUNICI...|2020|        366|
|99495199999|SEBASTIAN INLET S...|2020|        365|
|72429793812|CINCINNATI MUNICI...|2021|        365|
|99495199999|SEBASTIAN INLET S...|2021|        104|
|72429793812|CINCINNATI MUNICI...|2022|        365|
|99495199999|SEBASTIAN INLET S...|2022|        259|
|72429793812|CINCINNATI MUNICI...|2023|        365|
|99495199999

In [4]:
csv_df.filter(csv_df.MAX != 9999.9).createOrReplaceTempView("WEATHER_DATA")
queried_data = spark.sql("""
    SELECT
        STATION,
        LOCATION,
        DATE,
        MAX_TEMP
    FROM (
        SELECT
            WEATHER_DATA.STATION,
            WEATHER_DATA.NAME AS LOCATION,
            WEATHER_DATA.DATE,
            WEATHER_DATA.MAX AS MAX_TEMP,
            ROW_NUMBER() OVER (PARTITION BY WEATHER_DATA.YEAR ORDER BY WEATHER_DATA.MAX DESC) AS ranked_order
        FROM WEATHER_DATA
    ) AS max_temps
    WHERE max_temps.ranked_order = 1
""")
queried_data.show()

+-----------+--------------------+----------+--------+
|    STATION|            LOCATION|      DATE|MAX_TEMP|
+-----------+--------------------+----------+--------+
|72429793812|CINCINNATI MUNICI...|2015-06-12|    91.9|
|72429793812|CINCINNATI MUNICI...|2016-07-24|    93.9|
|72429793812|CINCINNATI MUNICI...|2017-07-22|    91.9|
|72429793812|CINCINNATI MUNICI...|2018-07-04|    96.1|
|72429793812|CINCINNATI MUNICI...|2019-09-30|    95.0|
|72429793812|CINCINNATI MUNICI...|2020-07-05|    93.9|
|72429793812|CINCINNATI MUNICI...|2021-08-12|    95.0|
|72429793812|CINCINNATI MUNICI...|2022-06-14|    96.1|
|72429793812|CINCINNATI MUNICI...|2023-08-23|    96.1|
|72429793812|CINCINNATI MUNICI...|2024-08-30|   100.9|
+-----------+--------------------+----------+--------+



In [5]:
csv_df.filter(csv_df.MIN != 9999.9).createOrReplaceTempView("WEATHER_DATA")
queried_data = spark.sql("""
    SELECT
        STATION,
        NAME AS LOCATION,
        DATE,
        MIN AS MIN_TEMP
    FROM WEATHER_DATA
    INNER JOIN (
        SELECT
            MIN(MIN) AS MIN_TEMP
        FROM WEATHER_DATA
        WHERE MONTH = 3
    ) AS min_data ON min_data.MIN_TEMP = WEATHER_DATA.MIN
""")
queried_data.show()

+-----------+--------------------+----------+--------+
|    STATION|            LOCATION|      DATE|MIN_TEMP|
+-----------+--------------------+----------+--------+
|72429793812|CINCINNATI MUNICI...|2015-03-06|     3.2|
+-----------+--------------------+----------+--------+



In [6]:
csv_df.filter(csv_df.PRCP != 99.99).createOrReplaceTempView("WEATHER_DATA")
queried_data = spark.sql("""
    SELECT
        STATION,
        LOCATION,
        YEAR,
        PRECIPITATION
    FROM (
        SELECT
            WEATHER_DATA.STATION,
            WEATHER_DATA.NAME AS LOCATION,
            WEATHER_DATA.YEAR,
            AVG(WEATHER_DATA.PRCP) AS PRECIPITATION,
            ROW_NUMBER() OVER (PARTITION BY WEATHER_DATA.STATION ORDER BY AVG(WEATHER_DATA.PRCP) DESC, WEATHER_DATA.YEAR) AS ranked_order
        FROM WEATHER_DATA
        GROUP BY WEATHER_DATA.YEAR, WEATHER_DATA.STATION, WEATHER_DATA.NAME
        ORDER BY WEATHER_DATA.YEAR, WEATHER_DATA.STATION
    ) AS avg_prcps
    WHERE avg_prcps.ranked_order = 1
    ORDER BY STATION
""")
queried_data.show()

+-----------+--------------------+----+-------------------+
|    STATION|            LOCATION|YEAR|      PRECIPITATION|
+-----------+--------------------+----+-------------------+
|72429793812|CINCINNATI MUNICI...|2018|0.15789041095890405|
|99495199999|SEBASTIAN INLET S...|2015|                0.0|
+-----------+--------------------+----+-------------------+



In [7]:
queried_data = spark.sql("""
    SELECT
        WEATHER_DATA.STATION,
        WEATHER_DATA.NAME AS LOCATION,
        (SUM(
            CASE
                WHEN WEATHER_DATA.GUST = 999.9 THEN 1
                ELSE 0
            END
        ) / COUNT(*)) * 100 AS PERCENT_MISSING_GUSTS
    FROM WEATHER_DATA
    WHERE WEATHER_DATA.YEAR = 2024
    GROUP BY WEATHER_DATA.STATION, WEATHER_DATA.NAME
    ORDER BY STATION
""")
queried_data.show()

+-----------+--------------------+---------------------+
|    STATION|            LOCATION|PERCENT_MISSING_GUSTS|
+-----------+--------------------+---------------------+
|72429793812|CINCINNATI MUNICI...|    39.14285714285714|
|99495199999|SEBASTIAN INLET S...|                100.0|
+-----------+--------------------+---------------------+



In [8]:
queried_data = spark.sql("""
    SELECT
        MONTH,
        AVG(TEMP) AS MEAN,
        MEDIAN(TEMP) AS MEDIAN,
        MODE(TEMP) AS MODE,
        STDDEV_POP(TEMP) AS STDEV
    FROM WEATHER_DATA
    WHERE STATION = 72429793812 AND YEAR = 2020
    GROUP BY MONTH
    ORDER BY MONTH
""")
queried_data.show()

+-----+------------------+------+------+------------------+
|MONTH|              MEAN|MEDIAN|  MODE|             STDEV|
+-----+------------------+------+------+------------------+
|    1| 37.94516129032259|  37.7|  24.7| 8.210097587321375|
|    2|  36.5896551724138|  36.0|  30.8| 7.764168131721883|
|    3|  49.0741935483871|  47.8|  53.2| 8.636642408457773|
|    4|51.779999999999994|  51.1|  53.2| 7.190243389482725|
|    5| 60.89032258064518|  63.7|  73.9| 9.163298280630869|
|    6| 72.54666666666667| 73.95|  74.2| 4.817588147149521|
|    7|              77.6|  77.9|  79.7| 2.299929872703694|
|    8| 73.34516129032258|  73.7|  73.2| 3.431151288931137|
|    9|              66.1| 66.15|  54.7|  6.99861891137577|
|   10|55.193548387096776|  54.0|  59.4| 6.619274664671979|
|   11|48.003333333333345|  47.7|  47.7| 6.711208700541371|
|   12| 35.99354838709677|  35.2|  37.4|6.5347673396336425|
+-----+------------------+------+------+------------------+



In [9]:
csv_df = csv_df.withColumn('WIND_CHILL', when((col('TEMP') < 50) & (col('WDSP') > 3), 35.74 + 0.6215 * col('TEMP') - 35.75 * col('WDSP')**0.16 + 0.4275 * col('TEMP') * col('WDSP')**0.16).otherwise(9999.9))
csv_df.filter(csv_df.TEMP != 9999.9).filter(csv_df.WDSP != 999.9).createOrReplaceTempView("WEATHER_DATA")

queried_data = spark.sql("""
    SELECT
        DATE,
        WIND_CHILL
    FROM WEATHER_DATA
    WHERE STATION = 72429793812 AND YEAR = 2017
    ORDER BY WIND_CHILL, DATE
    LIMIT 10
""")
queried_data.show()

+----------+-------------------+
|      DATE|         WIND_CHILL|
+----------+-------------------+
|2017-01-07|-0.4140156367932173|
|2017-12-31| 2.0339767075993116|
|2017-12-27|  3.820645509123832|
|2017-12-28|  4.533355269061226|
|2017-01-06|  4.868933041653884|
|2017-01-08|  7.929748208036862|
|2017-12-25| 14.285113218297408|
|2017-12-30| 14.539211253038193|
|2017-01-05| 14.748861828163854|
|2017-12-26| 15.688977805634499|
+----------+-------------------+



In [10]:
queried_data = spark.sql("""
    SELECT
        STATION,
        NAME AS LOCATION,
        COUNT(FRSHTT) AS DAYS_EXTREME
    FROM WEATHER_DATA
    WHERE FRSHTT <> 0 AND STATION = 99495199999
    GROUP BY STATION, LOCATION
""")
queried_data.show()

+-------+--------+------------+
|STATION|LOCATION|DAYS_EXTREME|
+-------+--------+------------+
+-------+--------+------------+



In [11]:
queried_data = spark.sql("""
    SELECT
        TEMP
    FROM WEATHER_DATA
    WHERE STATION = 72429793812 AND MONTH = 11 AND YEAR IN (2022, 2023)
    ORDER BY DATE
""").collect()
november_temps = [float(row.__getitem__('TEMP')) for row in queried_data]
queried_data = spark.sql("""
    SELECT
        TEMP
    FROM WEATHER_DATA
    WHERE STATION = 72429793812 AND MONTH = 12 AND YEAR IN (2022, 2023)
    ORDER BY DATE
""").collect()
december_temps = [float(row.__getitem__('TEMP')) for row in queried_data]
nov_22 = november_temps[0:int(len(november_temps)/2)]
nov_23 = november_temps[int(len(november_temps)/2):]
dec_22 = december_temps[0:int(len(december_temps)/2)]
dec_23 = december_temps[int(len(december_temps)/2):]

sum_yearly_diffs = 0
for i in range(len(dec_22)):
    if i < len(nov_22):
        diff = nov_23[i] - nov_22[i]
        sum_yearly_diffs = sum_yearly_diffs + diff
    diff = dec_23[i] - dec_22[i]
    sum_yearly_diffs = sum_yearly_diffs + diff

avg_diff = sum_yearly_diffs/(len(nov_22) + len(dec_22))

predicted_data = []
for i in range(len(nov_23)):
    predicted_data.append((f"11/{i + 1}/2024", nov_23[i] + avg_diff))
for i in range(len(dec_23)):
    predicted_data.append((f"12/{i + 1}/2024", dec_23[i] + avg_diff))

print('Date | Temperature')
for predicted_temp in predicted_data:
    print(f'{predicted_temp[0]} | {round(predicted_temp[1], 2)} F')

Date | Temperature
11/1/2024 | 41.76 F
11/2/2024 | 40.96 F
11/3/2024 | 45.86 F
11/4/2024 | 57.26 F
11/5/2024 | 53.86 F
11/6/2024 | 53.16 F
11/7/2024 | 70.06 F
11/8/2024 | 61.76 F
11/9/2024 | 68.46 F
11/10/2024 | 56.76 F
11/11/2024 | 46.46 F
11/12/2024 | 45.66 F
11/13/2024 | 46.96 F
11/14/2024 | 44.66 F
11/15/2024 | 46.96 F
11/16/2024 | 51.46 F
11/17/2024 | 60.06 F
11/18/2024 | 49.66 F
11/19/2024 | 43.16 F
11/20/2024 | 44.16 F
11/21/2024 | 52.76 F
11/22/2024 | 48.76 F
11/23/2024 | 42.86 F
11/24/2024 | 39.46 F
11/25/2024 | 39.06 F
11/26/2024 | 42.56 F
11/27/2024 | 39.16 F
11/28/2024 | 30.26 F
11/29/2024 | 30.96 F
11/30/2024 | 41.56 F
12/1/2024 | 52.06 F
12/2/2024 | 55.46 F
12/3/2024 | 51.66 F
12/4/2024 | 42.06 F
12/5/2024 | 42.46 F
12/6/2024 | 43.86 F
12/7/2024 | 41.86 F
12/8/2024 | 51.56 F
12/9/2024 | 60.26 F
12/10/2024 | 46.66 F
12/11/2024 | 38.86 F
12/12/2024 | 36.96 F
12/13/2024 | 35.96 F
12/14/2024 | 35.56 F
12/15/2024 | 38.26 F
12/16/2024 | 40.96 F
12/17/2024 | 52.16 F
12/18/2024 |

In [12]:
queried_data = spark.sql("""
    SELECT
        DATE,
        TEMP
    FROM WEATHER_DATA
    WHERE STATION = 72429793812 AND YEAR = 2024 AND MONTH IN (11, 12)
    ORDER BY DATE
""")
queried_data.show(n=queried_data.count())

+----------+------+
|      DATE|  TEMP|
+----------+------+
|2024-11-01|  55.3|
|2024-11-02|  46.0|
|2024-11-03|  50.7|
|2024-11-04|  65.4|
|2024-11-05|  69.9|
|2024-11-06|  68.8|
|2024-11-07|  60.5|
|2024-11-08|  49.6|
|2024-11-09|  49.1|
|2024-11-10|  58.5|
|2024-11-11|  57.0|
|2024-11-12|  47.0|
|2024-11-13|  44.4|
|2024-11-14|  53.8|
|2024-11-15|  52.2|
|2024-11-16|  50.6|
|2024-11-17|  46.3|
|2024-11-18|  57.4|
|2024-11-19|  59.7|
|2024-11-20|  52.8|
|2024-11-21|  38.2|
|2024-11-22|  37.9|
|2024-11-23|  44.9|
|2024-11-24|  46.2|
|2024-11-25|  51.2|
|2024-11-26|  46.6|
|2024-11-27|  41.5|
|2024-11-28|  41.4|
|2024-11-29|  34.1|
|2024-11-30|  27.0|
|2024-12-01|  31.1|
|2024-12-02|  26.3|
|2024-12-03|  25.9|
|2024-12-04|  32.6|
|2024-12-05|  29.0|
|2024-12-06|  23.6|
|2024-12-07|  31.0|
|2024-12-08|  47.6|
|2024-12-09|  54.4|
|2024-12-10|  48.0|
|2024-12-11|  35.6|
|2024-12-12|  28.5|
|2024-12-13|  27.6|
|2024-12-14|  34.2|
|2024-12-15|  46.2|
|2024-12-16|  52.9|
|2024-12-17|  46.8|
